# 2차 전처리후 모델링(캐글 제출)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

In [3]:
data = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')

In [4]:
data.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [5]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [6]:
data = reduce_mem_usage(data)

Memory usage of dataframe is 983.90 MB
Memory usage after optimization is: 288.39 MB
Decreased by 70.7%


In [7]:
data = data.dropna()

In [8]:
data = data.drop(columns=['Id','groupId', 'matchId', 'rankPoints', 'roadKills', 'matchDuration', 'teamKills', 'killPoints', 'winPoints', 'maxPlace'])

data.head(2)

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,killStreaks,longestKill,matchType,numGroups,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,winPlacePerc
0,0,0,0.0,0,0,0,60,0,0,0.0,squad-fpp,26,0,0.000000,0.000000,0,244.75,1,0.444336
1,0,0,91.5,0,0,0,57,0,0,0.0,squad-fpp,25,0,0.004501,11.039062,0,1434.00,5,0.640137


In [9]:
# matchType2라는 칼럼을 생성
data['matchType2'] = pd.NaT

# row가 많아서 생기는 메모리 에러 무시하고 진행
# pd.set_option('mode.chainedassignment',  None)

## crash와 flare를 event mode로 처리할 때    채택
maplist = {
    'squad' : 'squad',
    'duo' : 'duo',
    'solo' : 'solo',
    'crash' : 'event',
    'flare' : 'event'
}

for i in maplist:
  data.matchType2[(data['matchType'].str.contains(i)==True)&(data['matchType2'].isnull())]=maplist.get(i)


# solo ⮕ 1, duo ⮕ 2, squad ⮕ 4, falre & crash ⮕ 0 으로 전환
def convert(x):
    if x == "squad":
        return 4
    elif x == "duo":
        return 2
    elif x == "solo":
        return 1
    else : 
        return 0

data['matchType2'] = data['matchType2'].map(convert)


# 기존의 matchType drop
data.drop(['matchType'], axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [10]:
data['cat_numGroups'] = 0

data.loc[data['numGroups'] < 40, 'cat_numGroups'] = 0
data.loc[(data['numGroups'] > 40) & (data['numGroups'] < 80), 'cat_numGroups'] = 1
data.loc[data['numGroups'] > 80, 'cat_numGroups'] = 2

data = data.drop(columns='numGroups')

In [11]:
data = data.drop(columns = 'killStreaks')
data.head(2)

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,winPlacePerc,matchType2,cat_numGroups
0,0,0,0.0,0,0,0,60,0,0.0,0,0.000000,0.000000,0,244.75,1,0.444336,4,0
1,0,0,91.5,0,0,0,57,0,0.0,0,0.004501,11.039062,0,1434.00,5,0.640137,4,0


## Training

In [12]:
df = data

In [13]:
# 학습을 위한 training / test dataset 나누기
from sklearn.model_selection import train_test_split

X_num = df.drop(columns = ['matchType2', 'cat_numGroups', 'winPlacePerc'])
X_cat = df[['matchType2', 'cat_numGroups']]
y = df.winPlacePerc

In [14]:
# 스케일러
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num)
X_scaled = pd.DataFrame(data = X_scaled, index=X_num.index, columns = X_num.columns)

X = pd.concat([X_scaled, X_cat], axis=1)

In [15]:
# 트테트테
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
                                            
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(3557572, 17) (889393, 17) (3557572,) (889393,)


In [16]:
X

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,matchType2,cat_numGroups
0,-0.397257,-0.645129,-0.765409,-0.574191,-0.376680,-0.511252,0.451541,-0.593402,-0.451176,-0.348730,-0.404524,-0.147836,-0.085499,-0.768458,-1.083021,4,0
1,-0.397257,-0.645129,-0.229634,-0.574191,-0.376680,-0.511252,0.342303,-0.593402,-0.451176,-0.348730,-0.404521,0.214073,-0.085499,0.236403,0.545283,4,0
2,1.301767,-0.645129,-0.367238,-0.574191,-0.376680,-0.511252,-0.021824,-0.593402,-0.451176,-0.348730,-0.404524,-0.147836,-0.085499,-0.838589,-0.675945,2,1
3,-0.397257,-0.645129,-0.572728,-0.574191,-0.376680,-0.511252,0.997732,-0.593402,-0.451176,-0.348730,-0.404524,-0.147836,-0.085499,-0.803946,-0.268869,4,0
4,-0.397257,-0.645129,-0.179863,-0.574191,-0.376680,-0.511252,-0.094650,0.048264,0.697113,-0.348730,-0.404524,-0.147836,-0.085499,-0.933224,-0.675945,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446961,-0.397257,-0.645129,-0.765409,-0.574191,-0.376680,-0.511252,0.961319,-0.593402,-0.451176,-0.348730,0.457762,-0.147836,-0.085499,-0.114253,-0.268869,4,0
4446962,-0.397257,-0.062308,-0.506854,-0.574191,-0.376680,-0.511252,0.779256,-0.593402,-0.451176,-0.348730,-0.404524,-0.147836,-0.085499,-0.906239,0.952359,1,2
4446963,-0.397257,-0.645129,-0.419571,-0.574191,-0.376680,-0.511252,0.670017,-0.593402,-0.451176,-0.348730,-0.404524,-0.076248,-0.085499,-0.309015,0.138207,4,0
4446964,-0.397257,1.686154,0.290770,0.298605,1.284022,0.235021,-1.332682,0.689929,1.481235,3.887058,-0.404524,-0.147836,-0.085499,1.346672,1.766511,4,0


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from xgboost.sklearn import XGBRegressor
from lightgbm.sklearn import LGBMRegressor

from sklearn.metrics import mean_absolute_error

In [18]:
reg5 = LGBMRegressor()
reg5.fit(X_train, y_train)
pred_train5 = reg5.predict(X_train)
pred_val5 = reg5.predict(X_val)
mse_train5 = mean_absolute_error(y_train, pred_train5)
mse_val5 = mean_absolute_error(y_val, pred_val5)
print("5. LightGBM\t\t, train, test = %.4f, %.4f" % (mse_train5, mse_val5))

5. LightGBM		, train, test = 0.0682, 0.0682


## Test

In [19]:
test = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')

In [20]:
test = reduce_mem_usage(test)

Memory usage of dataframe is 413.18 MB
Memory usage after optimization is: 121.74 MB
Decreased by 70.5%


In [21]:
test = test.drop(columns=['groupId', 'matchId', 'rankPoints', 'roadKills', 'matchDuration', 'teamKills', 'killPoints', 'winPoints', 'maxPlace'])

test.head(2)

,Id,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,killStreaks,longestKill,matchType,numGroups,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired
0,9329eb41e215eb,0,0,51.46875,0,0,0,73,0,0,0.0,squad-fpp,28,0,0.0,0.0,0,588.0,1
1,639bd0dcd7bda8,0,4,179.12500,0,0,2,11,2,1,362.0,duo-fpp,47,2,4668.0,0.0,0,2017.0,6


In [22]:
# matchType2라는 칼럼을 생성
test['matchType2'] = pd.NaT

# row가 많아서 생기는 메모리 에러 무시하고 진행
# pd.set_option('mode.chainedassignment',  None)

## crash와 flare를 event mode로 처리할 때    채택
maplist = {
    'squad' : 'squad',
    'duo' : 'duo',
    'solo' : 'solo',
    'crash' : 'event',
    'flare' : 'event'
}

for i in maplist:
  test.matchType2[(test['matchType'].str.contains(i)==True)&(test['matchType2'].isnull())]=maplist.get(i)


# solo ⮕ 1, duo ⮕ 2, squad ⮕ 4, falre & crash ⮕ 0 으로 전환
def convert(x):
    if x == "squad":
        return 4
    elif x == "duo":
        return 2
    elif x == "solo":
        return 1
    else : 
        return 0

test['matchType2'] = test['matchType2'].map(convert)


# 기존의 matchType drop
test.drop(['matchType'], axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [23]:
test['cat_numGroups'] = 0

test.loc[test['numGroups'] < 40, 'cat_numGroups'] = 0
test.loc[(test['numGroups'] > 40) & (test['numGroups'] < 80), 'cat_numGroups'] = 1
test.loc[test['numGroups'] > 80, 'cat_numGroups'] = 2

test = test.drop(columns='numGroups')

In [24]:
test.columns

Index(['Id', 'assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills',
       'heals', 'killPlace', 'kills', 'killStreaks', 'longestKill', 'revives',
       'rideDistance', 'swimDistance', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'matchType2', 'cat_numGroups'],
      dtype='object')

In [25]:
X_num_test = test.drop(columns = ['matchType2', 'cat_numGroups','Id'])
X_cat_test = test[['matchType2', 'cat_numGroups']]

X_num_test

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,killStreaks,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired
0,0,0,51.468750,0,0,0,73,0,0,0.0000,0,0.0,0.000000,0,588.000,1
1,0,4,179.125000,0,0,2,11,2,1,362.0000,2,4668.0,0.000000,0,2017.000,6
2,1,0,23.406250,0,0,4,49,0,0,0.0000,0,0.0,0.000000,0,788.000,4
3,0,0,65.500000,0,0,0,54,0,0,0.0000,0,0.0,0.000000,0,1812.000,3
4,0,4,330.250000,1,2,1,7,3,1,60.0625,1,0.0,0.000000,0,2964.000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934169,1,2,381.000000,3,0,7,4,4,2,69.2500,1,0.0,0.000000,0,1799.000,5
1934170,0,0,0.000000,0,0,0,64,0,0,0.0000,0,0.0,0.000000,0,1195.000,3
1934171,0,0,91.937500,0,0,3,24,1,1,25.3750,0,0.0,7.796875,0,3328.000,3
1934172,1,2,138.625000,0,0,12,25,1,1,67.8750,1,0.0,0.000000,0,1893.000,4


In [26]:
X_scaled_test = scaler.fit_transform(X_num_test)
X_scaled_test

array([[-0.39743462, -0.6443822 , -0.46746984, ..., -0.08394702,
        -0.47577652, -1.1138382 ],
       [-0.39743462,  1.6895753 ,  0.29494387, ..., -0.08394702,
         0.735582  ,  1.0056667 ],
       [ 1.3308996 , -0.6443822 , -0.63507026, ..., -0.08394702,
        -0.30623716,  0.1578648 ],
       ...,
       [-0.39743462, -0.6443822 , -0.22577445, ..., -0.08394702,
         1.8469123 , -0.26603618],
       [ 1.3308996 ,  0.52259654,  0.05306182, ..., -0.08394702,
         0.63046765,  0.1578648 ],
       [-0.39743462, -0.6443822 , -0.6566269 , ..., -0.08394702,
        -0.91583705, -1.5377392 ]], dtype=float32)

In [27]:
X_scaled_test = pd.DataFrame(data = X_scaled_test, index=X_num_test.index, columns = X_num_test.columns)
X_test = pd.concat([X_scaled_test, X_cat_test], axis=1)
X_test

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,killStreaks,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,matchType2,cat_numGroups
0,-0.397435,-0.644382,-0.467470,-0.577010,-0.380549,-0.509436,0.917096,-0.602691,-0.766681,-0.451597,-0.348157,-0.400698,-0.148872,-0.083947,-0.475777,-1.113838,4,0
1,-0.397435,1.689575,0.294944,-0.577010,-0.380549,0.240838,-1.340508,0.709851,0.643734,6.706196,3.923213,2.764269,-0.148872,-0.083947,0.735582,1.005667,2,1
2,1.330900,-0.644382,-0.635070,-0.577010,-0.380549,0.991112,0.043185,-0.602691,-0.766681,-0.451597,-0.348157,-0.400698,-0.148872,-0.083947,-0.306237,0.157865,4,0
3,-0.397435,-0.644382,-0.383670,-0.577010,-0.380549,-0.509436,0.225250,-0.602691,-0.766681,-0.451597,-0.348157,-0.400698,-0.148872,-0.083947,0.561804,-0.266036,2,1
4,-0.397435,1.689575,1.197522,0.305066,2.989906,-0.134299,-1.486160,1.366122,0.643734,0.736014,1.787528,-0.400698,-0.148872,-0.083947,1.538351,0.157865,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934169,1.330900,0.522597,1.500621,2.069216,-0.380549,2.116522,-1.595399,2.022393,2.054149,0.917677,1.787528,-0.400698,-0.148872,-0.083947,0.550784,0.581766,4,0
1934170,-0.397435,-0.644382,-0.774862,-0.577010,-0.380549,-0.509436,0.589379,-0.602691,-0.766681,-0.451597,-0.348157,-0.400698,-0.148872,-0.083947,0.038775,-0.266036,4,0
1934171,-0.397435,-0.644382,-0.225774,-0.577010,-0.380549,0.615975,-0.867139,0.053580,0.643734,0.050141,-0.348157,-0.400698,0.108767,-0.083947,1.846912,-0.266036,4,0
1934172,1.330900,0.522597,0.053062,-0.577010,-0.380549,3.992207,-0.830727,0.053580,0.643734,0.890489,1.787528,-0.400698,-0.148872,-0.083947,0.630468,0.157865,2,1


In [28]:
# X_test = X_test.drop('Id', axis = 1)
# X_test

In [29]:
X_test = X_test.drop(columns = 'killStreaks')
X_test.head(2)

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,matchType2,cat_numGroups
0,-0.397435,-0.644382,-0.467470,-0.57701,-0.380549,-0.509436,0.917096,-0.602691,-0.451597,-0.348157,-0.400698,-0.148872,-0.083947,-0.475777,-1.113838,4,0
1,-0.397435,1.689575,0.294944,-0.57701,-0.380549,0.240838,-1.340508,0.709851,6.706196,3.923213,2.764269,-0.148872,-0.083947,0.735582,1.005667,2,1


In [30]:
X_test.columns

Index(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPlace', 'kills', 'longestKill', 'revives', 'rideDistance',
       'swimDistance', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired',
       'matchType2', 'cat_numGroups'],
      dtype='object')

In [31]:
X.columns

Index(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPlace', 'kills', 'longestKill', 'revives', 'rideDistance',
       'swimDistance', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired',
       'matchType2', 'cat_numGroups'],
      dtype='object')

In [32]:
result5 = reg5.predict(X_test)

In [33]:
test['Id']

0          9329eb41e215eb
1          639bd0dcd7bda8
2          63d5c8ef8dfe91
3          cf5b81422591d1
4          ee6a295187ba21
                ...      
1934169    a316c3a13887d5
1934170    5312146b27d875
1934171    fc8818b5b32ad3
1934172    a0f91e35f8458f
1934173    3696fc9f3a42b2
Name: Id, Length: 1934174, dtype: object

In [34]:
submission = pd.DataFrame({'Id' : test.Id,'winPlacePerc' : result5})
submission.to_csv('submission.csv',index=False)
print(submission)

                     Id  winPlacePerc
0        9329eb41e215eb      0.234680
1        639bd0dcd7bda8      0.915528
2        63d5c8ef8dfe91      0.659081
3        cf5b81422591d1      0.625872
4        ee6a295187ba21      0.906798
...                 ...           ...
1934169  a316c3a13887d5      0.738592
1934170  5312146b27d875      0.399706
1934171  fc8818b5b32ad3      0.835041
1934172  a0f91e35f8458f      0.799247
1934173  3696fc9f3a42b2      0.069971

[1934174 rows x 2 columns]
